In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def preprocessing(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['build_year'] = np.where(df['build_year'] < 1500.0, np.nan, df['build_year'])
    df['build_year'] = np.where(df['build_year'] > 2017.0, np.nan, df['build_year'])
    df['build_year'] = 'Year_' + df['build_year'].astype(str)
    df['build_year_flg'] = np.where(pd.isnull(df['build_year']), 'N', 'Y')
    df['state'] = np.where(df['state'] > 4.0, np.nan, df['state'])
    df['floor'] = np.where(df['floor'] < 10.0, np.nan, df['floor'])
    df['num_room'] = np.where(df['num_room'] > 9.0, np.nan, df['num_room'])
    #df['material'] = np.where(pd.isnull(df['material']), 7.0, df['material'])
    df['material'] = 'Material_' + df['material'].astype(str)
    #df['state'] = np.where(pd.isnull(df['state']), 5, df['state'])
    df['state'] = 'State_' + df['state'].astype(str)
    df['state_flg'] = np.where(pd.isnull(df['state']), 'N', 'Y')
    df['hospital_beds_raion_flg'] = np.where(pd.isnull(df['hospital_beds_raion']), 'N', 'Y')
    df['cafe_500_flg'] = np.where(pd.isnull(df['cafe_sum_500_min_price_avg']), 'N', 'Y')
    df['floor_flg'] = np.where(pd.isnull(df['max_floor']), 'N', 'Y')
    df['preschool_quota_flg'] = np.where(pd.isnull(df['preschool_quota']), 'N', 'Y')
    df['school_quota_flg'] = np.where(pd.isnull(df['school_quota']), 'N', 'Y')
    df['cafe_1000_flg'] = np.where(pd.isnull(df['cafe_sum_1000_min_price_avg']), 'N', 'Y')
    df['life_sq_flg'] = np.where(pd.isnull(df['life_sq']), 'N', 'Y')
    df['build_flg'] = np.where(pd.isnull(df['raion_build_count_with_material_info']), 'N', 'Y')
    df['cafe_2000_flg'] = np.where(pd.isnull(df['cafe_sum_2000_min_price_avg']), 'N', 'Y')
    df['cafe_3000_flg'] = np.where(pd.isnull(df['cafe_sum_3000_min_price_avg']), 'N', 'Y')
    df['cafe_5000_flg'] = np.where(pd.isnull(df['cafe_sum_5000_min_price_avg']), 'N', 'Y')
    df['metro_flg'] = np.where(pd.isnull(df['metro_min_walk']), 'N', 'Y')
    df['prom_part_5000'] = np.where(pd.isnull(df['prom_part_5000']), 'N', 'Y')
    df['floor_flg'] = np.where(pd.isnull(df['floor']), 'N', 'Y')

    df['life_sq'] = np.where(df['life_sq'] > df['full_sq'], np.nan, df['life_sq'])
    df['kitch_sq'] = np.where(df['kitch_sq'] > df['full_sq'], np.nan, df['kitch_sq'])
    df['max_floor'] = np.where(df['floor'] > df['max_floor'], df['floor'], df['max_floor'])

    df['year'] = pd.DatetimeIndex(df['timestamp']).year
    df['month'] = pd.DatetimeIndex(df['timestamp']).month
    df['day'] = pd.DatetimeIndex(df['timestamp']).day
    df['weekday'] = pd.DatetimeIndex(df['timestamp']).weekday
    object = []
    for i in df.columns:
        if (df[i].dtype == 'object'): object.append(i)

    #dummies = pd.get_dummies(df[object])
    number = LabelEncoder()
    for i in object:
        df[i] = number.fit_transform(df[i].astype(str))
    #df = df.drop(object, axis=1)
    #df = pd.concat([df, dummies], axis=1)
    imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)  ## trying with Most frequent
    ddfwotime = pd.to_numeric(df['timestamp'])
    df = df.drop('timestamp', axis=1)
    df_complete = pd.DataFrame(imp.fit_transform(df), columns= df.columns)
    #df_complete = df
    df_complete['timestamp'] = ddfwotime
    return df_complete

In [ ]:
def rmsle(y, y_pred):
	assert len(y) == len(y_pred)
	terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
	return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(480, input_dim=480, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sklearn.preprocessing as st
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
import math
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import fancyimpute as fc

In [ ]:
df = pd.read_csv("../input/train.csv",sep=',')
test = pd.read_csv("../input/test.csv",sep=',')
macro = pd.read_csv("../input/macro.csv",sep=',')

In [ ]:
macro['child_on_acc_pre_school'] = macro['child_on_acc_pre_school'].str.replace(",","")
macro['old_education_build_share'] = macro['old_education_build_share'].str.replace(",",".")
macro['modern_education_share'] = macro['modern_education_share'].str.replace(",",".")

In [ ]:
macro['timestamp'] = pd.to_datetime(macro['timestamp'])
macro['child_on_acc_pre_school']= pd.to_numeric(macro['child_on_acc_pre_school'],errors='coerce')
macro['old_education_build_share']= pd.to_numeric(macro['old_education_build_share'],errors='coerce')
macro['modern_education_share']= pd.to_numeric(macro['modern_education_share'],errors='coerce')

In [ ]:
print(macro[['modern_education_share','timestamp']].groupby(['modern_education_share']).agg(['count']))

In [ ]:
object =[]
for i in macro.columns:
    if(macro[i].dtype =='object'):
        lst = []
        print(i)
        object.append(i)
        lst.append(i)
        lst.append('timestamp')
        print(macro[lst].groupby([i]).agg(['count']))

In [ ]:
print (object)

In [ ]:
macro_wotimestamp = macro.drop(['timestamp'], axis = 1)
tk = pd.DataFrame.as_matrix(macro_wotimestamp)

In [ ]:
X_filled_knn = fc.KNN(k=3).complete(tk)

In [ ]:
varnames = macro_wotimestamp.columns.tolist()

In [ ]:
print (varnames)

In [ ]:
macro_fill = pd.DataFrame(X_filled_knn, columns=varnames)
macro_fill['timestamp'] = macro['timestamp']

In [ ]:
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [ ]:
macro = macro_fill

In [ ]:
np.sum(np.sum(pd.isnull(macro)))

In [ ]:
timestam = pd.to_numeric(macro['timestamp'])

In [ ]:

macro['timestamp'] =timestam

In [ ]:
macro.head(3)

In [ ]:
df_complete = preprocessing(df)
test_complete = preprocessing(test)
df_complete_train = pd.merge(df_complete,macro,how ='left' ,on=['timestamp'])
test_complete = pd.merge(test_complete,macro,how ='left' ,on=['timestamp'])

In [ ]:
target = df_complete_train.columns.tolist().index('price_doc')
id = df_complete_train.columns.tolist().index('id')
noofcol = len(df_complete_train.columns)
r =  list(range(0,noofcol))
del r[target]
del r[id]
features = df_complete_train.columns[r]
data = df_complete_train.values
#X_filled_knn = fc.KNN(k=3).complete(data)
#data = X_filled_knn
Y = data[:,target]
y_log = np.log(data[:,target])
X = data[:,r]

In [ ]:
np.sum(np.isnan(X))

In [ ]:
testid = test_complete.columns.tolist().index('id')
testcol = len(test_complete.columns)
r =  list(range(0,testcol))
del r[id]
Test_features = test_complete.columns[r]
testdata = test_complete.values
#X_filled_knn = fc.KNN(k=3).complete(testdata)
#testdata = X_filled_knn
testX = testdata[:,r]
#end

In [ ]:
np.sum(np.isnan(testX))

In [ ]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=test_size, random_state=seed)

estimators = []
gbm = xg.XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=.7, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=.7)
estimators.append(('standardize', st.RobustScaler()))
estimators.append(('mlp', gbm))
pipeline = Pipeline(estimators)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
res = rmsle(np.exp(y_test),np.exp(y_pred))

In [ ]:
print (res)

In [ ]:
testy_pred = pipeline.predict(testX)
pred_ytest = np.exp(testy_pred)
ids = test_complete['id']

results = pd.DataFrame(columns=list(['id','price_doc']))
results['id'] = ids
results['id'] = results['id'].astype(int)
results['price_doc'] = pred_ytest
results.to_csv("Results.csv",index=False)

In [ ]:
rt = pd.read_csv("Results.csv",sep=",")


In [ ]:
print(rt)